In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('raw_data.csv')

In [3]:
data.shape

(10422, 100)

In [4]:
drop_none=['PEFAMID','PAS1MRG1','PAS1MRG2','PWARPPG','PWATOTPG','PWNETWPG','VERDATE']
data.drop(columns=drop_none, inplace=True)

In [5]:
data.shape

(10422, 93)

DATA CLEANING

1. Real Estate

In [6]:
# 1 - Create D_PASRBUYG': Home owner IN Canada? (32)
data['D_PASRBUYG']=data['PASRBUYG']
data['D_PASRBUYG']=np.where((data['D_PASRBUYG']!=96),1,0)
print((data[data['D_PASRBUYG']==1]['PWEIGHT'].sum()/data['PWEIGHT'].sum()*100).round(4),'% of households own a home in CANADA')

62.4058 % of households own a home in CANADA


In [7]:
# 1 - Recode to years (32)
data['PASRBUYG']=np.where(data['PASRBUYG']==96, (2019-2019),
np.where(data['PASRBUYG']==1, (2019-1960),
np.where(data['PASRBUYG']==2, (2019-1970),
np.where(data['PASRBUYG']==3, (2019-1980),
np.where(data['PASRBUYG']==4, (2019-1990),
np.where(data['PASRBUYG']==5, (2019-2000),  
np.where(data['PASRBUYG']==6, (2019-2005),
np.where(data['PASRBUYG']==7, (2019-2010),
np.where(data['PASRBUYG']==8, (2019-2015),
np.where(data['PASRBUYG']==9, (2019-2017), 1))))))))))


In [8]:
#(pd.DataFrame(data['PASRBUYG'].value_counts())).sort_index()

In [9]:
# 2 - Create D_PASRCNMG: A dummy variable for 'OTHER' real estate IN Canada (34)
data['D_PASRCNMG']=data['PASRCNMG']
data['D_PASRCNMG']=np.where(data['D_PASRCNMG']==6,0,1)
print((data[data['D_PASRCNMG']==1]['PWEIGHT'].sum()/data['PWEIGHT'].sum()*100).round(2),'% of households have OTHER real estate IN Canada')

13.12 % of households have OTHER real estate IN Canada


In [10]:
# 2.1 - Recode missings to zeroes (34)
data['PASRCNMG'].replace(to_replace=[6], value=[0],inplace=True)
data['PASRCNMG'].max()

4

In [11]:
# 3 - Create D_PASRFNMG: dummy for those who have a property OUTSIDE of Canada / Recoding (53)
data['PASRFNMG'].replace(to_replace=[6],value=[0], inplace=True)

data['D_PASRFNMG']=data['PASRFNMG']
data['D_PASRFNMG']=np.where(data['D_PASRFNMG']==0,0,1)

print((pd.DataFrame(data['D_PASRFNMG'].value_counts())).sort_index())
print((pd.DataFrame(data['PASRFNMG'].value_counts())).sort_index())

   D_PASRFNMG
0        9877
1         545
   PASRFNMG
0      9877
1       427
2       118


In [12]:
# 4 - Recode: Class collapse (35)
data['PASRCON'].replace(to_replace=[9], value=[2], inplace=True)
print((pd.DataFrame(data['PASRCON'].value_counts())).sort_index())

   PASRCON
1     1045
2     9377


In [13]:
# 5 - Impute: zeroes to those who did not purchase a house (36) / drop original variable
data['PASRCST'].replace(to_replace=[99999999], value=[99999996], inplace=True)
data['IMP_PASRCST']=data['PASRCST']
data['IMP_PASRCST'].replace(to_replace=[99999996], value=[0], inplace=True)
data.drop(columns=['PASRCST'], inplace=True)
print('New mean:',data['IMP_PASRCST'].mean())
print('Max:',data.IMP_PASRCST.max())
print('PASRCST has been dropped')

New mean: 193677.29658414892
Max: 3800000
PASRCST has been dropped


In [14]:
# 6 - Recode variable to percentages (51)
data['PASRDWNG'].replace(to_replace=[96], value=[0], inplace=True)
data['PASRDWNG']=np.where(data['PASRDWNG']==0, 0,
np.where(data['PASRDWNG']==1, 5,
np.where(data['PASRDWNG']==2, 15,
np.where(data['PASRDWNG']==3, 25,
np.where(data['PASRDWNG']==4, 35,
np.where(data['PASRDWNG']==5, 45,  
np.where(data['PASRDWNG']==6, 55,
np.where(data['PASRDWNG']==7, 65,
np.where(data['PASRDWNG']==8, 90, 100)))))))))

In [15]:
#(pd.DataFrame(data['PASRDWNG'].value_counts())).sort_index()

In [16]:
# 7 - Recode variable - Class collapse (64)
data['PASRRNTG'].replace(to_replace=[9], value=[6], inplace=True)

In [17]:
# 8 - D_PFTENUR: Dummy for homeowners with no mortgage (138)
data['D_PFTENUR']=data['PFTENUR']
data['D_PFTENUR']=np.where(data['D_PFTENUR']==1,1,0)
data['D_PFTENUR'].value_counts()

0    6743
1    3679
Name: D_PFTENUR, dtype: int64

In [18]:
data.shape

(10422, 97)

2. Mortgage

In [19]:
# 1 - Create D_PASR1MR: A dummy mortgage variable (31); also look to 138
data['D_PASR1MR']=data['PASR1MR']
data['D_PASR1MR']=np.where(data['D_PASR1MR']==6,0,1)
print((data[data['D_PASR1MR']==1]['PWEIGHT'].sum()/data['PWEIGHT'].sum()*100).round(2),'% of households have at least one mortgage in Canada')

34.83 % of households have at least one mortgage in Canada


In [20]:
# 2 - Create IMP_PEXMG1A: Zeroes are imputed to those who dont have a mortgage (123)
data['IMP_PEXMG1A']=data['PEXMG1A']
data.loc[data['IMP_PEXMG1A']==99996,['IMP_PEXMG1A']]=0
print('Households ith a 99996 code:',data[data['IMP_PEXMG1A']==99996]['IMP_PEXMG1A'].count())
print('(ALL) Households (INCLUDING THOSE THAT DONT HAVE MORTGAGES) make an average payment of:',data.IMP_PEXMG1A.mean())
data.drop(columns=['PEXMG1A'], inplace=True)
print('PEXMG1A has been dropped')

Households ith a 99996 code: 0
(ALL) Households (INCLUDING THOSE THAT DONT HAVE MORTGAGES) make an average payment of: 367.60650546919976
PEXMG1A has been dropped


In [21]:
# 3 - Mortgage payment recode (128)
data['PEXMG1F'].replace(to_replace=[6],value=[0], inplace=True)
data['PEXMG1F'].replace(to_replace=[5],value=[0], inplace=True)
data['PEXMG1F'].replace(to_replace=[4],value=[2], inplace=True)
code=[0,1,2,4]
mort_payment_freq=['Skip','Monthly','Every two weeks','Weekly']
pd.DataFrame({'New Code':code,'Payment Frequency':mort_payment_freq, 'Count':data['PEXMG1F'].value_counts().sort_index()})

,New Code,Payment Frequency,Count
0,0,Skip,6707
1,1,Monthly,1413
2,2,Every two weeks,1870
3,4,Weekly,432


In [22]:
# 4.1 - Create D_PASR1MFA': A dummy variable for mortgage payments on additional property(23)
data['D_PASR1MFA']=data['PASR1MFA']
data['D_PASR1MFA']=np.where((data.D_PASR1MFA!=99999996) & (data.D_PASR1MFA>0),1,0)
print((data[data['D_PASR1MFA']==1]['PWEIGHT'].sum()/data['PWEIGHT'].sum()*100).round(2),'% of households made mortgage payments on additional properties')

8.01 % of households made mortgage payments on additional properties


In [23]:
# 4.2 - Creating a IMP_PASR1MFA: Zeroes are imputed to those who skipped the question (23) - 
# Do you a mortgage? NO - Skip
# Do you have a mortgage on additional properties? - YES; How much do you pay? $ X amount of dollars
# Do you have a mortgage on additional properties? - NO; How much do you pay? $ 0 dollars
data['IMP_PASR1MFA']=data['PASR1MFA']
data.loc[data['IMP_PASR1MFA']==99999996,['IMP_PASR1MFA']]=0
data.drop(columns=['PASR1MFA'], inplace=True)
print('Households with a 99999996 code:',data[data['IMP_PASR1MFA']==99999996]['IMP_PASR1MFA'].count())
print('(ALL) Households (INCLUDING THOSE THAT DONT HAVE MORTGAGES ON ADDITIONAL PROPERTIES) make a ADDITIONAL average mortgage payment of:',data.IMP_PEXMG1A.mean())
print('PASR1MFA has been dropped')

Households with a 99999996 code: 0
(ALL) Households (INCLUDING THOSE THAT DONT HAVE MORTGAGES ON ADDITIONAL PROPERTIES) make a ADDITIONAL average mortgage payment of: 367.60650546919976
PASR1MFA has been dropped


In [24]:
# 5 Creating a D_PAS1MRAG: Did you refinance your mortgage? (20)
data['D_PAS1MRAG']=np.where(data['PAS1MRAG']==6,0,1)
print(data['D_PAS1MRAG'].value_counts())
data.drop(columns=['PAS1MRAG'],inplace=True)
print('PAS1MRAG has been dropped')

0    10186
1      236
Name: D_PAS1MRAG, dtype: int64
PAS1MRAG has been dropped


In [25]:
# 6 - Recode skips to 0 and turn codes into years (44)
data['PASRCURG'].replace(to_replace=[96], value=[0], inplace=True)
data['PASRCURG'].replace(to_replace=[8], value=[0], inplace=True)
data['PASRCURG']=np.where(data['PASRCURG']==0,0,
np.where(data['PASRCURG']==1,0.6,
np.where(data['PASRCURG']==2,1,
np.where(data['PASRCURG']==3,2,
np.where(data['PASRCURG']==4,3,
np.where(data['PASRCURG']==5,4,
np.where(data['PASRCURG']==6,5,10)))))))
(pd.DataFrame(data['PASRCURG'].value_counts())).sort_index()

,PASRCURG
0.0,6864
0.6,44
1.0,163
2.0,195
3.0,597
4.0,191
5.0,2264
10.0,104


In [26]:
# 7 - Recoding interest rates (55)
data['PASRINTG']=np.where(data['PASRINTG']==96,0,
np.where(data['PASRINTG']==1,1.99,
np.where(data['PASRINTG']==2,2.49,
np.where(data['PASRINTG']==3,2.99,
np.where(data['PASRINTG']==4,3.49,
np.where(data['PASRINTG']==5,3.99,
np.where(data['PASRINTG']==6,4.99,6)))))))
(pd.DataFrame(data['PASRINTG'].value_counts())).sort_index()

,PASRINTG
0.00,6697
1.99,51
2.49,366
2.99,1287
3.49,1108
3.99,530
4.99,244
6.00,139


In [27]:
# 8 - Recoding interest terms codes into years (57)
data['PASRMOAG']=np.where(data['PASRMOAG']==6,0,
np.where(data['PASRMOAG']==1,14,
np.where(data['PASRMOAG']==2,24,
np.where(data['PASRMOAG']==3,29,40))))
(pd.DataFrame(data['PASRMOAG'].value_counts())).sort_index()

,PASRMOAG
0,6697
14,128
24,708
29,2298
40,591


In [28]:
# 9 - Recoding codes into number of years reaminig in the mortgage (58)
data['PASRMPFG']=np.where(data['PASRMPFG']==96,0,
np.where(data['PASRMPFG']==1,5,
np.where(data['PASRMPFG']==2,9,
np.where(data['PASRMPFG']==3,14,
np.where(data['PASRMPFG']==4,19,
np.where(data['PASRMPFG']==5,24,30))))))
(pd.DataFrame(data['PASRMPFG'].value_counts())).sort_index()

,PASRMPFG
0,6697
5,343
9,389
14,467
19,825
24,1213
30,488


In [29]:
# 10 - Recoding codes into number of years remaining for renewal (60)
data['PASRMRYG']=np.where(data['PASRMRYG']==9996,0,
np.where(data['PASRMRYG']==2019,(2019-2018),
np.where(data['PASRMRYG']==2020,(2020-2018),
np.where(data['PASRMRYG']==2021,(2021-2018),
np.where(data['PASRMRYG']==2022,(2022-2018),
np.where(data['PASRMRYG']==2023,(2023-2018),6))))))
(pd.DataFrame(data['PASRMRYG'].value_counts())).sort_index()

,PASRMRYG
0,6697
1,159
2,823
3,807
4,739
5,625
6,572


In [30]:
data.shape

(10422, 99)

3. Credit & Credit Card

In [31]:
# 1- Recode: impute zeroes to skipped values (86) - There's already a dummy: have lines of credit? (86)
data['PATTLMLC'].replace(to_replace=[99999996], value=[0], inplace=True)
data['PATTLMLC'].skew()

5.517677265777087

In [32]:
# 2- Recode: Imputing zeroes to skipped values (76)
data['PATTLCP'].replace(to_replace=[6], value=[0], inplace=True)

In [33]:
# 3 - Recode: Imputing zeroes to skipped values (67) - There's already a dummy: have credit cards?
data['PATTCRLM'].replace(to_replace=[999996], value=[0], inplace=True)
data.PATTCRLM.skew()

7.353505973637755

In [34]:
# 4 - Recoding skips to zeroes (74)
data['PATTCRU'].replace(to_replace=[6],value=[0], inplace=True)

In [35]:
# 4.1 - Creating a D_PATTCRU: Did you pay the minimum amount or less? (74)
data['D_PATTCRU']=data['PATTCRU']
for i in data['D_PATTCRU']:
    if i==2:
        data['D_PATTCRU'].replace(to_replace=[i], value=[1], inplace=True)
    elif i==3:
        data['D_PATTCRU'].replace(to_replace=[i], value=[1], inplace=True)
    elif i==4:
        data['D_PATTCRU'].replace(to_replace=[i], value=[0], inplace=True)
    elif i==5:
        data['D_PATTCRU'].replace(to_replace=[i], value=[0], inplace=True)
data.D_PATTCRU.value_counts()

0    7220
1    3202
Name: D_PATTCRU, dtype: int64

In [36]:
# 5 - Recoding skips into number of credit cards (129)
for i in data['PFCRN']:
    if i==4:
        data['PFCRN'].replace(to_replace=[i], value=[0], inplace=True)
    elif i==1:
        data['PFCRN'].replace(to_replace=[i], value=[1.5], inplace=True)
    elif i==2:
        data['PFCRN'].replace(to_replace=[i], value=[3.5], inplace=True)
    elif i==3:
        data['PFCRN'].replace(to_replace=[i], value=[5.5], inplace=True)
data.PFCRN.value_counts()

1.5    5203
3.5    2983
5.5    1290
0.0     946
Name: PFCRN, dtype: int64

In [37]:
data.shape

(10422, 100)

4. RRSP

5. INCOME

In [38]:
# 1.1 - Creating D1_PEFGTR: Do you receive government transfers? (105)
data['D1_PEFGTR']=data['PEFGTR']
for i in data['D1_PEFGTR']:
    if i>0:
        data['D1_PEFGTR'].replace(to_replace=[i],value=[1],inplace=True)
    elif i==0:
        data['D1_PEFGTR'].replace(to_replace=[i],value=[0],inplace=True)
data['D1_PEFGTR'].value_counts()

1    9340
0    1082
Name: D1_PEFGTR, dtype: int64

In [39]:
# 1.2 - Creating D2_PEFGTR: is household in the last decile of amount of transfer income? (105)
data['D2_PEFGTR']=data['PEFGTR']
data['D2_PEFGTR']=np.where(data['D2_PEFGTR']>=data['D2_PEFGTR'].quantile(.9),1,0)
data['D2_PEFGTR'].value_counts()

0    9318
1    1104
Name: D2_PEFGTR, dtype: int64

In [40]:
# 2 - Recoding Not Stated to the mode (151)
data['PNBEARG'].replace(to_replace=[9], value=[1],inplace=True)

In [41]:
# 3.1 - Creating D_PINHERT: Did you receive an inheritance?
data['D_PINHERT']=data['PINHERT']
data['D_PINHERT']=np.where(data['D_PINHERT']==999999996,0,1)
data['D_PINHERT'].value_counts()

0    6985
1    3437
Name: D_PINHERT, dtype: int64

In [42]:
# 3.2 - Creating IMP_PINHERT: Impute zeroes to those who didnt recieve an inheritance
data['IMP_PINHERT']=data['PINHERT']
data['IMP_PINHERT'].replace(to_replace=[999999996],value=[0], inplace=True)
data[data['IMP_PINHERT']==999999996]['IMP_PINHERT'].count()

0

In [43]:
# 3.2.1 - Drop PINHERT
data.drop(columns=['PINHERT'], inplace=True)

In [44]:
# 4 - Creating D_PEFATINC: is household in the first decile of after tax income?
data['D_PEFATINC']=data['PEFATINC']
data['D_PEFATINC']=np.where(data['D_PEFATINC']<=data['D_PEFATINC'].quantile(.1),1,0)
data['D_PEFATINC'].value_counts()

0    9379
1    1043
Name: D_PEFATINC, dtype: int64

In [45]:
# 5.1 - Creating IMP_PEFMTINC: Impute zeroes to market income
data['IMP_PEFMTINC']=data['PEFMTINC']
data['IMP_PEFMTINC'].replace(to_replace=[99999999],value=[0], inplace=True)
data[data['IMP_PEFMTINC']==99999999]['IMP_PEFMTINC'].count()

0

In [46]:
# 5.2 - Creating D_PEFATINC: is household in the first decile of before tax income? (115)
data['D_PEFMTINC']=data['PEFMTINC']
data['D_PEFMTINC']=np.where(data['D_PEFMTINC']<=data['D_PEFMTINC'].quantile(.1),1,0)
data['D_PEFMTINC'].value_counts()

0    9370
1    1052
Name: D_PEFMTINC, dtype: int64

In [47]:
data.drop(columns=['PEFMTINC'],inplace=True)

In [48]:
data.shape

(10422, 105)

6. FAMILY COMPOSITION

In [49]:
# 1 - Recode Not Stated to the mode (131)
data['PFSZ'].replace(to_replace=[9], value=[2], inplace=True)

In [50]:
data.shape

(10422, 105)

7. MAJOR EARNER

In [51]:
# 1 - Recoding to ages of major income earners (18)
for i in data['PAGEMIEG']:
    if i==1:
        data['PAGEMIEG'].replace(to_replace=[i],value=[19],inplace=True)
    elif i==2:
        data['PAGEMIEG'].replace(to_replace=[i],value=[24],inplace=True)
    elif i==3:
        data['PAGEMIEG'].replace(to_replace=[i],value=[29],inplace=True)
    elif i==4:
        data['PAGEMIEG'].replace(to_replace=[i],value=[34],inplace=True)
    elif i==5:
        data['PAGEMIEG'].replace(to_replace=[i],value=[39],inplace=True)
    elif i==6:
        data['PAGEMIEG'].replace(to_replace=[i],value=[44],inplace=True)
    elif i==7:
        data['PAGEMIEG'].replace(to_replace=[i],value=[49],inplace=True)
    elif i==8:
        data['PAGEMIEG'].replace(to_replace=[i],value=[54],inplace=True)
    elif i==9:
        data['PAGEMIEG'].replace(to_replace=[i],value=[59],inplace=True)
    elif i==10:
        data['PAGEMIEG'].replace(to_replace=[i],value=[64],inplace=True)
    elif i==11:
        data['PAGEMIEG'].replace(to_replace=[i],value=[69],inplace=True)
    elif i==12:
        data['PAGEMIEG'].replace(to_replace=[i],value=[74],inplace=True)
    elif i==13:
        data['PAGEMIEG'].replace(to_replace=[i],value=[79],inplace=True)
    elif i==14:
        data['PAGEMIEG'].replace(to_replace=[i],value=[84],inplace=True)

In [52]:
#(pd.DataFrame(data['PAGEMIEG'].value_counts())).sort_index()

In [53]:
# 2 - Recode not stated to skip (149)
data['PLFFPTME'].replace(to_replace=[9], value=[6], inplace=True)

In [54]:
data.shape

(10422, 105)

8. FUNDS

In [55]:
fund_var=['PWAOTPEN','PWARPPT','PWASTBND','PWARRIF','PWARRSPL','PWASTDEP','PWASTMUI',
'PWASTOIN','PWASTONF','PWASTSTK','PWATFS']

In [56]:
for i in data[fund_var]:
    data['D_'+i]=data[i]
    data['D_'+i]=np.where(data['D_'+i]>1,1,0)

In [57]:
data['PWASTSTK'].replace(to_replace=[0],value=[1],inplace=True)

In [58]:
data.shape

(10422, 116)

9. DEBT

In [59]:
# 1 Creating D_PWDTOTAL: Has debts? (457)
data['D_PWDTOTAL']=data['PWDTOTAL']
data['D_PWDTOTAL']=np.where(data['D_PWDTOTAL']>0,1,0)
data['D_PWDTOTAL'].value_counts()

1    7322
0    3100
Name: D_PWDTOTAL, dtype: int64

In [60]:
# 2 - Creating D_PWDSLOAN: Has student loans? (385)
data['D_PWDSLOAN']=data['PWDSLOAN']
data['D_PWDSLOAN']=np.where(data['D_PWDSLOAN']>0,1,0)
data['D_PWDSLOAN'].value_counts()

0    9302
1    1120
Name: D_PWDSLOAN, dtype: int64

In [61]:
# 3 - Creating D_PWDSTODB: Has debts with other financial institutions? (421)
data['D_PWDSTODB']=data['PWDSTODB']
data['D_PWDSTODB']=np.where(data['D_PWDSTODB']>0,1,0)
data['D_PWDSTODB'].value_counts()

0    9510
1     912
Name: D_PWDSTODB, dtype: int64

In [62]:
# 4 - Creating D_PWDSTVHN: Has debts on vehicles? (445)
data['D_PWDSTVHN']=data['PWDSTVHN']
data['D_PWDSTVHN']=np.where(data['D_PWDSTVHN']>0,1,0)
data['D_PWDSTVHN'].value_counts()

0    7157
1    3265
Name: D_PWDSTVHN, dtype: int64

In [63]:
data.shape

(10422, 120)

10. ASSETS

In [64]:
# 1 - Creating D_PWASTVHE: Has vehicles? (313)
data['D_PWASTVHE']=data['PWASTVHE']
data['D_PWASTVHE']=np.where(data['D_PWASTVHE']>0,1,0)
data['D_PWASTVHE'].value_counts()

1    8888
0    1534
Name: D_PWASTVHE, dtype: int64

In [65]:
# 2 - Creating D_PWATOTPT: is in the first decile?
data['D_PWATOTPT']=data['PWATOTPT']
data['D_PWATOTPT']=np.where(data['D_PWATOTPT']<=data['D_PWATOTPT'].quantile(.1),1,0)
data['D_PWATOTPT'].value_counts()

0    9379
1    1043
Name: D_PWATOTPT, dtype: int64

In [66]:
data.shape

(10422, 122)

11. NET WORTH

In [67]:
# 1 - Creating D_PWNETWPT: Has negative equity?
data['D1_PWNETWPT']=data['PWNETWPT']
data['D1_PWNETWPT']=np.where(data['D1_PWNETWPT']<0,1,0)
data['D1_PWNETWPT'].value_counts()

0    9941
1     481
Name: D1_PWNETWPT, dtype: int64

In [68]:
data['D2_PWNETWPT']=data['PWNETWPT']
data['D2_PWNETWPT']=np.where(data['D2_PWNETWPT']<=data['D2_PWNETWPT'].quantile(.1),1,0)
data['D2_PWNETWPT'].value_counts()

0    9377
1    1045
Name: D2_PWNETWPT, dtype: int64

In [69]:
data.shape

(10422, 124)

In [70]:
data.rename(columns={'PFS17':'PFSZ0017'}, inplace=True)

In [71]:
data.to_csv('pre_eda_data.csv')